
# Frage 1 – Notebook 02: Aufbereitung & Normalisierung

Dieses Notebook
- lädt die in Notebook 01 erzeugte CSV `warenkorb_auswertung.csv`,
- filtert die **letzten 5 Jahre**,
- liest die Umfrage (`umfrage.xlsx`, Spalten **AN** und **AO**) ein,
- normalisiert die Mehrfachantworten (Aspekte) und
- speichert bereinigte Zwischenstände als CSV für die nächsten Notebooks.


In [4]:

import pandas as pd
import numpy as np
from pathlib import Path

# Projekt-Root ermitteln (anpassen falls nötig)
project_root = Path("D:/JupyterProject")

# Datenordner
data_path = project_root / "data"
out_path = project_root / "Frage1"  # für konsistente Ablage
out_path.mkdir(exist_ok=True)

csv_path = "warenkorb_auswertung.csv"
df = pd.read_csv(csv_path)
df["Jahr"] = pd.to_numeric(df["Jahr"], errors="coerce")

# Auf die letzten 5 Jahre beschränken
last_year = int(df["Jahr"].max())
window = list(range(last_year - 4, last_year + 1))
df5 = df[df["Jahr"].isin(window)].sort_values("Jahr").reset_index(drop=True)
df5.to_csv(out_path / "warenkorb_auswertung_5J.csv", index=False)
df5.head()


,Jahr,CPI,LM_Umsatz_nom,Bekl_nom,Food_nom,LM_Umsatz_real,Bekl_real,Food_real
0,2016,121.448574,182192.08,77.71,217.54,150015.825355,63.985931,179.121083
1,2017,123.270302,193640.36,80.57,224.22,157085.978221,65.360430,181.892959
2,2018,125.489168,197272.60,79.97,232.57,157202.891515,63.726616,185.330738
3,2019,127.246016,216323.77,77.93,240.22,170004.356048,61.243568,188.783907
4,2020,127.882246,224559.32,73.65,257.71,175598.511118,57.592044,201.521328


## Umfragedaten einlesen & normalisieren

In [5]:

# Umfrage (AN: Veränderung_Warenkorb, AO: Aspekte)
umf = pd.read_excel(data_path / "Umfrage.xlsx", usecols="AN:AO",
                    names=["Veränderung_Warenkorb", "Aspekte"])

# Verteilung Veränderung
order_ver = ["Deutlich gestiegen","Etwas gestiegen","Gleich geblieben","Etwas gesunken","Deutlich gesunken"]
ver = (umf["Veränderung_Warenkorb"].value_counts().reindex(order_ver).fillna(0)
       .rename_axis("Antwort").reset_index(name="Anzahl"))
ver["Anteil_%"] = (ver["Anzahl"] / len(umf) * 100).round(1)
ver.to_csv(out_path / "umfrage_warenkorb_verteilung.csv", index=False)
ver


,Antwort,Anzahl,Anteil_%
0,Deutlich gestiegen,11,28.2
1,Etwas gestiegen,15,38.5
2,Gleich geblieben,9,23.1
3,Etwas gesunken,3,7.7
4,Deutlich gesunken,1,2.6


In [6]:

# Aspekte (Mehrfachauswahl, Semikolon-separiert)
aspekte = (umf["Aspekte"].fillna("").str.split(";", expand=False).explode().str.strip())
aspekte = aspekte[aspekte.ne("")]

order_aspekte = [
    "Höhere Produktqualität / Markenprodukte",
    "Verbesserter Kundenservice / Rückgabeservice",
    "Lokale / europäische Anbieter statt Billiganbieter",
    "Nachhaltige oder umweltfreundliche Produkte",
    "Schnellerer Versand / Expresslieferung",
    "Keine erhöhte Zahlungsbereitschaft",
    "Sonstiges"
]
asp = (aspekte.value_counts().reindex(order_aspekte).fillna(0)
       .rename_axis("Aspekt").reset_index(name="Anzahl"))
asp["Anteil_%"] = (asp["Anzahl"] / len(umf) * 100).round(1)
asp.to_csv(out_path / "umfrage_warenkorb_aspekte.csv", index=False)
asp


,Aspekt,Anzahl,Anteil_%
0,Höhere Produktqualität / Markenprodukte,29.0,74.4
1,Verbesserter Kundenservice / Rückgabeservice,15.0,38.5
2,Lokale / europäische Anbieter statt Billiganbi...,16.0,41.0
3,Nachhaltige oder umweltfreundliche Produkte,14.0,35.9
4,Schnellerer Versand / Expresslieferung,8.0,20.5
5,Keine erhöhte Zahlungsbereitschaft,2.0,5.1
6,Sonstiges,0.0,0.0
